<h1>Programación matemática</h1>
<p>Este modelo genera las cantidades sugeridas a pedir un único proveedor, que puede atender a varias bodegas, cada una de ellas con su propio forecast de venta en función del forecast de venta.</p>
<p><strong>Conjuntos</strong>:</p>
<ul>
    <li>b: conjunto de bodegas</li>
    <li>t: conjunto de periodos</li>
</ul>
<p><strong>Parámetros</strong>:</p>
<ul>
    <li>$ HE $: costo unitario de mantener el producto en bodega. Surge del costo del producto y el costo de oportunidad por periodo</li>
    <li>$ CP $: costo de colocar una orden de abastecimiento para un sku totalizando las tres bodegas</li>
    <li>$ I_{bo} $: Inventario inicial en la bodega b</li>
    <li>$ SS_{b} $: Inventario de seguridad para el final del periodo. El modelo intentará mantener el cierre de inventario sobre esta cifra en on_hand</li>
    <li>$ LM $: Lote mínimo de producción, asegurará que la suma del pedido de las bodegas no esté por debajo de este valor.</li>
    <li>$ MPO $: Pedido mínimo: Asegura que no se sugieran pedidos de menos de este valor.</li>
    <li>$ F_{bt} $: Pronóstico de ventas del producto en la bodega b durante el periodo t</li>
    <li>$ A_{bt} $: Llegadas calculadas de producto a la bodega b durante el periodo t</li>
    <li>$ LT_{b} $: Lead time calculado entre las plantas y las bodegas</li>
</ul>
<p><strong>Variables</strong>:</p>
<ul>
    <li>$ X_{bt} $: Cantidad de producto a pedir para la bodega b durante el periodo t</li>
    <li>$ M_{bt} $: Binaria, 1 si se coloca el pedido mínimo (30 metros?) durante el periodo t para la bodega b; 0 en otr caso</li>
    <li>$ P_{t} $: Binary, 1 si se coloca un pedido desde cualquiera de las bodegas durante el periodo t. Esta se usará para garantizar el pedido mínimo para producción</li>
    <li>$ I_{bt} $: Variable de Estado, Cantidad de producto al cierre del periodo t en la bodega b</li>
</ul>
<p><strong>Función objetivo</strong>:</p>
<p>La siguiente expresión minimiza el costo total relevante compuesto por el costo total de colocar las órdenes de pedido y el costo de mantener el inventario:</p>
<p>$ min \sum_{bt}{CP*P_{t} + HE*X_{bt}} $ </p>
<p><strong>Sujeto a</strong>:</p>
<p><strong>Resticción de Inventario de seguridad:</strong> Evita que quede inventario por debajo del SS luego del leadtime</p>
<p>$ I_{bt} \geq {SS_{bt}} \forall{t > LT_{b}}$</p>
<p><strong>Flujos de inventario:</strong> Mantiene el balance de masa del inventario. Para el primer caso, se encarga de inicializar el inventario, en el segundo del flujo durante el lead time y, finalmente, el flujo cuando los pedidos colocados empiezan a llegar</p>
<p>$ I_{bt} = I_{bo} - F_{bt} + A_{bt} \forall{b}; t=0 $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} \forall b ; \forall 1 < t < LT $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} + X_{bt} \forall b ; \forall t \geq LT $</p>

In [1]:
from lp_supply.inventory_solvers import mono_product_multiperiod_multi_location
import numpy as np

<p>Una compañia dispone de 3 bodegas en 3 lugares diferentes, cada una de ellas con una demanda de un producto establecida, un inventario inicial, un inventario de seguridad. La siguiente tabla muestra la demanda de producto de las tres bodegas:</p>
<table>
    <tr>
        <td>Variable</td>
        <td>Semana 1</td>
        <td>Semana 2</td>
        <td>Semana 4</td>
        <td>Semana 5</td>
        <td>Semana 6</td>
    </tr>
    <tr>
        <td>Demanda bodega 1</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
    </tr>
    <tr>
        <td>Demanda bodega 2</td>
        <td>20</td>
        <td>10</td>
        <td>0</td>
        <td>10</td>
        <td>10</td>
    </tr>
    <tr>
        <td>Demanda bodega 3</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
        <td>10</td>
    </tr>
</table>
<p>Suponga que las tres bodegas tienen un safety stock de 10 unidades, el pedido mínimo que se puede hacer al proveedor es de 400 unidades, y cada bodega debe recibir al menos 50 unidades. Si no hay pedidos anteriores puestos al proveedor (arrivals en 0) y todas las bodegas tienen un lead time de 1 semana, cuál debe ser el plan de pedidos?

In [2]:
# crear una variable donde la función va a retornar los valores
ejemplo = mono_product_multiperiod_multi_location(holding_cost=0.46,
                                                  order_cost=150,
                                                  initial_inventory=[10.0, 0.0, 0.0],
                                                  safety_stock=[10,10,10],
                                                  min_batch_size=400,
                                                  min_po=50,
                                                  forecast=[[10,10,10,10,10],
                                                            [20,10,0,10,10],
                                                            [10,10,10,10,10]],
                                                  arrivals=[[0,0,0,0,0],
                                                            [0,0,0,0,0],
                                                            [0,0,0,0,0]],
                                                  leadtime=[1,1,1])


<p>Luego de ejecutar la función, los pedidos para las bodegas respectivas son:</p>

In [3]:
print('Pedidos para la bodega 1:', ejemplo[0][0])
print('Pedidos para la bodega 2:', ejemplo[0][1])
print('Pedidos para la bodega 3:', ejemplo[0][2])

Pedidos para la bodega 1: [150.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 2: [150.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 3: [100.0, 0.0, 0.0, 0.0, 0.0]


<p>EL invenario proyectado para el final de cada semana es:</p>

In [4]:
print('inventario para la bodega 1:', ejemplo[1][0])
print('inventario para la bodega 2:', ejemplo[1][1])
print('inventario para la bodega 3:', ejemplo[1][2])

inventario para la bodega 1: [0.0, 140.0, 130.0, 120.0, 110.0]
inventario para la bodega 2: [-20.0, 120.0, 120.0, 110.0, 100.0]
inventario para la bodega 3: [-10.0, 80.0, 70.0, 60.0, 50.0]


<p>Note que el inventario de las bodegas 2 y 3 cierra en la primera semana en negativo, esto es debido a que el inventario inicial es muy bajo y no alcanza a ser cubierto por el pedido se el modelo colocará en la primera semana.</p>
<p>A continuación, asumiremos que el pedido mínimo es de 150 unidades</p>

In [5]:
# crear una variable donde la función va a retornar los valores
ejemplo = mono_product_multiperiod_multi_location(holding_cost=0.46,
                                                  order_cost=150,
                                                  initial_inventory=[10.0, 0.0, 0.0],
                                                  safety_stock=[10,10,10],
                                                  min_batch_size=150,
                                                  min_po=50,
                                                  forecast=[[10,10,10,10,10],
                                                            [20,10,0,10,10],
                                                            [10,10,10,10,10]],
                                                  arrivals=[[0,0,0,0,0],
                                                            [0,0,0,0,0],
                                                            [0,0,0,0,0]],
                                                  leadtime=[1,1,1])


<p>Por lo que luego de ejecutar la función, los pedidos para las bodegas cambian, pero asegurando pedir mínimo 50 unidades para cada bodega y 150 en total al proveedor:</p>

In [6]:
print('Pedidos para la bodega 1:', ejemplo[0][0])
print('Pedidos para la bodega 2:', ejemplo[0][1])
print('Pedidos para la bodega 3:', ejemplo[0][2])

Pedidos para la bodega 1: [50.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 2: [60.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 3: [60.0, 0.0, 0.0, 0.0, 0.0]


<p>y el invenario proyectado para el final de cada semana es cambiará a:</p>

In [7]:
print('inventario para la bodega 1:', ejemplo[1][0])
print('inventario para la bodega 2:', ejemplo[1][1])
print('inventario para la bodega 3:', ejemplo[1][2])

inventario para la bodega 1: [0.0, 40.0, 30.0, 20.0, 10.0]
inventario para la bodega 2: [-20.0, 30.0, 30.0, 20.0, 10.0]
inventario para la bodega 3: [-10.0, 40.0, 30.0, 20.0, 10.0]


<p>Note que el inventario de las bodegas 2 y 3 continúa cerrando en la primera semana en negativo,
<p>finalmente, supongamos que la demanda de la bodega 3 en la semana 4 cambia a 50 unidades, por lo que re-calcularemos los pedidos:</p>

In [8]:
# crear una variable donde la función va a retornar los valores
ejemplo = mono_product_multiperiod_multi_location(holding_cost=0.46,
                                                  order_cost=150,
                                                  initial_inventory=[10.0, 0.0, 0.0],
                                                  safety_stock=[10,10,10],
                                                  min_batch_size=150,
                                                  min_po=50,
                                                  forecast=[[10,10,10,10,10],
                                                            [20,10,0,10,10],
                                                            [10,10,10,50,10]],
                                                  arrivals=[[0,0,0,0,0],
                                                            [0,0,0,0,0],
                                                            [0,0,0,0,0]],
                                                  leadtime=[1,1,1])


<p>Al ejecutar la función, los pedidos para las bodegas cambian, ahora la bodega 3 hará un pedido más grande y el modelo continua asegurando pedir mínimo 50 unidades para cada bodega y 150 en total al proveedor:</p>

In [9]:
print('Pedidos para la bodega 1:', ejemplo[0][0])
print('Pedidos para la bodega 2:', ejemplo[0][1])
print('Pedidos para la bodega 3:', ejemplo[0][2])

Pedidos para la bodega 1: [50.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 2: [60.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 3: [100.0, 0.0, 0.0, 0.0, 0.0]


<p>Vamos a cambiar el costo de colocar la orden de compra a 1. Esto supondrá que ahora es más costoso mantener el inventario en bodega que colocar pedidos al proveedor, por lo que el modelo deberá colocar más pedidos:</p>

In [10]:
# crear una variable donde la función va a retornar los valores
ejemplo = mono_product_multiperiod_multi_location(holding_cost=0.46,
                                                  order_cost=1,
                                                  initial_inventory=[10.0, 0.0, 0.0],
                                                  safety_stock=[10,10,10],
                                                  min_batch_size=150,
                                                  min_po=50,
                                                  forecast=[[10,10,10,10,10],
                                                            [20,10,0,10,10],
                                                            [10,10,10,50,10]],
                                                  arrivals=[[0,0,0,0,0],
                                                            [0,0,0,0,0],
                                                            [0,0,0,0,0]],
                                                  leadtime=[1,1,1])


<p>Note que ahora el el modelo coloca otro pedido en la semana 4 por lo que el inventario proyectado se verá ahora así:</p>

In [11]:
print('Pedidos para la bodega 1:', ejemplo[0][0])
print('Pedidos para la bodega 2:', ejemplo[0][1])
print('Pedidos para la bodega 3:', ejemplo[0][2])
print('inventario para la bodega 1:', ejemplo[1][0])
print('inventario para la bodega 2:', ejemplo[1][1])
print('inventario para la bodega 3:', ejemplo[1][2])

Pedidos para la bodega 1: [50.0, 0.0, 0.0, 0.0, 0.0]
Pedidos para la bodega 2: [50.0, 0.0, 0.0, 100.0, 0.0]
Pedidos para la bodega 3: [90.0, 0.0, 0.0, 50.0, 0.0]
inventario para la bodega 1: [0.0, 40.0, 30.0, 20.0, 10.0]
inventario para la bodega 2: [-20.0, 20.0, 20.0, 10.0, 100.0]
inventario para la bodega 3: [-10.0, 70.0, 60.0, 10.0, 50.0]
